In [59]:
from typing import TypedDict, Optional, Dict, Any, List
from enum import Enum
from langgraph.graph import StateGraph, END, START
from langgraph.checkpoint.memory import MemorySaver
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnableConfig
import json


In [48]:
from dotenv import load_dotenv
import getpass
import os

# Load environment variables from .env file
load_dotenv()

GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [49]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

E0000 00:00:1758548360.383720   22700 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


### Define the Application State

In [50]:
class ActivityLevel(str, Enum):
        sedentary = "sedentary"
        lightly_active = "lightly_active"
        moderately_active = "moderately_active"
        very_active = "very_active"
        extremely_active = "extremely_active"

class Gender(str, Enum):
    male = "Male"
    female = "Female"
    other = "Other"

class Goal(str, Enum):
    fat_loss = "Fat loss"
    muscle_build = "Muscle build"
    stay_active = "Stay active"


class FitnessAppState(TypedDict):
    # User Profile Information
    height: float
    weight: float
    age: int
    gender: Gender 
    activity_level: ActivityLevel 
    
    # User Goals
    goal_type: Goal 
    target_weight: float
    target_days: int
    
    # User Notes
    user_notes: Optional[str]
    
    # Generated Plans
    nutrition_plan: Optional[Dict[str, Any]]
    #*****************
    nutrition_plan_approved: Optional[bool]
    workout_plan: Optional[Dict[str, Any]]
    
    # Workflow Control
    current_step: str
    user_input_pending: bool
    edit_request: Optional[str]
    
    # Chat Context
    chat_messages: List[Dict[str, str | None]]
    chat_query: Optional[str]
    chat_response: Optional[str]

    # Error Handling
    error_message: Optional[str]

### Helper functions

In [51]:
def calculate_bmr(height: float, weight: float, age: int, gender: str) -> float:
    """Calculate Basal Metabolic Rate using Mifflin-St Jeor equation"""
    if gender.lower() == "male":
        return 10 * weight + 6.25 * height - 5 * age + 5
    else:
        return 10 * weight + 6.25 * height - 5 * age - 161

def calculate_daily_calories(bmr: float, activity_level: str) -> float:
    """Calculate daily calorie needs based on activity level"""
    multipliers = {
        "sedentary": 1.2,
        "lightly_active": 1.375,
        "moderately_active": 1.55,
        "very_active": 1.725,
        "extremely_active": 1.9
    }
    return bmr * multipliers.get(activity_level.lower(), 1.2)

def adjust_calories_for_goal(daily_calories: float, goal_type: str, target_weight: float, current_weight: float, target_days: int) -> float:
    """Adjust calories based on user goals"""
    if goal_type == "weight_loss":
        # Safe deficit: 0.5-1 kg per week = 500-1000 cal deficit per day
        weekly_loss = (current_weight - target_weight) / (target_days / 7)
        deficit = min(weekly_loss * 1000, 1000)  # Max 1000 cal deficit
        return daily_calories - deficit
    elif goal_type == "muscle_gain":
        return daily_calories + 300  # Moderate surplus
    else:
        return daily_calories  # Maintenance

### Node Functions

In [52]:
def collect_personal_info(state: FitnessAppState) -> FitnessAppState:
    """Node to collect user personal information"""
    print("📋 Please provide your personal information:")
    print("- Height (in cm)")
    print("- Weight (in kg)")
    print("- Age")
    print("- Gender (male/female/other)")
    print("- Activity Level (sedentary/lightly_active/moderately_active/very_active/extremely_active)")
    
    state["current_step"] = "personal_info"
    state["user_input_pending"] = True
    return state

def collect_goals(state: FitnessAppState) -> FitnessAppState:
    """Node to collect user goals"""
    print("🎯 Please provide your fitness goals:")
    print("- Goal Type (weight_loss/muscle_gain/maintenance/strength/endurance)")
    print("- Target Weight (in kg)")
    print("- Target Timeline (in days)")
    
    state["current_step"] = "goals"
    state["user_input_pending"] = True
    return state

def collect_notes(state: FitnessAppState) -> FitnessAppState:
    """Node to collect additional user notes"""
    print("📝 Please provide any additional notes (dietary restrictions, preferences, medical conditions, etc.):")
    print("This is optional but helps create a more personalized plan.")
    
    state["current_step"] = "notes"
    state["user_input_pending"] = True
    return state

In [53]:
def generate_nutrition_plan(state: FitnessAppState) -> FitnessAppState:
    """Node to generate personalized nutrition plan using LLM"""
    print("🍎 Generating your personalized nutrition plan...")
    
    # Calculate BMR and daily calorie needs
    bmr = calculate_bmr(state["height"], state["weight"], state["age"], state["gender"])
    daily_calories = calculate_daily_calories(bmr, state["activity_level"])
    
    # Adjust calories based on goal
    target_calories = adjust_calories_for_goal(
        daily_calories,
        state["goal_type"],
        state["target_weight"],
        state["weight"],
        state["target_days"]
    )
    
    # Nutrition prompt for LLM
    nutrition_prompt = f"""
    Create a detailed nutrition plan for a user with the following profile:
    - Age: {state['age']}, Gender: {state['gender']}
    - Height: {state['height']}cm, Weight: {state['weight']}kg
    - Activity Level: {state['activity_level']}
    - Goal: {state['goal_type']}, Target Weight: {state['target_weight']}kg in {state['target_days']} days
    - Target Daily Calories: {target_calories}
    - Additional Notes: {state['user_notes']}
    
    Provide the answer in valid JSON format with keys:
    - daily_calories
    - macros (protein, carbs, fats in grams)
    - meal_plan (detailed plan)
    - hydration
    - supplements
    """
    
    response = llm.invoke(nutrition_prompt)  
    
    # Try parsing JSON if model returns structured
    try:
        nutrition_plan = json.loads(response.content if hasattr(response, "content") else response)
    except Exception:
        # Fallback if model didn’t give strict JSON
        nutrition_plan = {"plan_text": response}
    
    # Save into state
    state["nutrition_plan"] = nutrition_plan
    state["current_step"] = "nutrition_review"
    state["user_input_pending"] = True
    return state

In [54]:
def generate_workout_plan(state: FitnessAppState) -> FitnessAppState:
    """Node to generate personalized workout plan"""
    print("💪 Generating your personalized workout plan...")
    
    workout_prompt = f"""
    Create a detailed workout plan for a user with the following profile:
    - Age: {state['age']}, Gender: {state['gender']}
    - Height: {state['height']}cm, Weight: {state['weight']}kg
    - Activity Level: {state['activity_level']}
    - Goal: {state['goal_type']}, Target Weight: {state['target_weight']}kg in {state['target_days']} days
    - Nutrition Plan: {state['nutrition_plan']}
    - Additional Notes: {state['user_notes']}
    
    Provide:
    1. Weekly workout schedule
    2. Exercise types and intensity
    3. Progressive overload plan
    4. Recovery recommendations
    5. Alternative exercises for equipment limitations

    Provide the answer in valid JSON format. Here is an example output:
        workout_plan = {
        "weekly_schedule": {
            "monday": "Upper body strength training",
            "tuesday": "Cardio - 30min moderate intensity",
            "wednesday": "Lower body strength training",
            "thursday": "Active recovery - yoga/stretching",
            "friday": "Full body circuit training",
            "saturday": "Cardio - 45min low intensity",
            "sunday": "Rest day"
        },
        "progression": "Weekly progression guidelines...",
        "recovery": "Recovery and rest recommendations..."
    }
    """
    
    response = llm.invoke(workout_prompt)  
    
    # Try parsing JSON if model returns structured
    try:
        nutrition_plan = json.loads(response.content if hasattr(response, "content") else response)
    except Exception:
        # Fallback if model didn’t give strict JSON
        nutrition_plan = {"plan_text": response}
    
    state["workout_plan"] = workout_plan
    state["current_step"] = "complete"
    return state


In [55]:
def handle_chat_query(state: FitnessAppState) -> FitnessAppState:
    """Node to handle general user queries in chat"""
    print(f"💬 Processing your question: {state['chat_query']}")
    
    chat_prompt = f"""
    User context:
    - Profile: Age {state['age']}, {state['gender']}, {state['height']}cm, {state['weight']}kg
    - Activity Level: {state['activity_level']}
    - Goals: {state['goal_type']}, target {state['target_weight']}kg in {state['target_days']} days
    - User Notes: {state['user_notes']}
    - Current Nutrition Plan: {state['nutrition_plan']}
    - Current Workout Plan: {state['workout_plan']}
    
    User Question: {state['chat_query']}
    
    Provide a helpful, personalized response considering their profile and plans.
    """
    
    response = llm.invoke(chat_prompt)
    
    # Add to chat history
    if not state["chat_messages"]:
        state["chat_messages"] = []
    
    state["chat_messages"].append({
        "user": state["chat_query"],
        "assistant": response
    })
    
    state["chat_response"] = response
    state["chat_query"] = None
    return state


### Conditional Edge Functions

In [56]:
def has_user_input_pending(state: FitnessAppState) -> str:
    """Check if waiting for user input"""
    if state.get("user_input_pending"):
        return "wait_for_input"
    else:
        return "continue"

def is_chat_query(state: FitnessAppState) -> str:
    """Check if this is a chat query"""
    if state.get("chat_query"):
        return "handle_chat"
    else:
        return "continue_workflow"

In [57]:
# Build the LangGraph
def create_fitness_app_graph():
    """Create and configure the LangGraph workflow"""
    
    # Initialize the graph
    workflow = StateGraph(FitnessAppState)
    
    # Add nodes
    workflow.add_node("collect_personal_info", collect_personal_info)
    workflow.add_node("collect_goals", collect_goals)
    workflow.add_node("collect_notes", collect_notes)
    workflow.add_node("generate_nutrition_plan", generate_nutrition_plan)
    workflow.add_node("generate_workout_plan", generate_workout_plan)
    workflow.add_node("handle_chat_query", handle_chat_query)
    
    # Set entry point
    workflow.add_edge(START, "collect_personal_info")
    
    # Add edges
    workflow.add_edge("collect_personal_info", "collect_goals")
    workflow.add_edge("collect_goals", "collect_notes")
    workflow.add_edge("collect_notes", "generate_nutrition_plan")
    workflow.add_edge("generate_nutrition_plan", "generate_workout_plan")
    
    # Chat functionality can be accessed from any completed state
    workflow.add_conditional_edges(
        "generate_workout_plan",
        is_chat_query,
        {
            "handle_chat": "handle_chat_query",
            "continue_workflow": END
        }
    )
    
    # Chat can loop back to itself or end
    workflow.add_conditional_edges(
        "handle_chat_query",
        is_chat_query,
        {
            "handle_chat": "handle_chat_query",
            "continue_workflow": END
        }
    )
    
    # Add memory for conversation persistence
    memory = MemorySaver()
    app = workflow.compile(checkpointer=memory)
    
    return app


In [60]:
# Example usage and testing
def main():
    """Main function to demonstrate the workflow"""
    
    # Create the application
    app = create_fitness_app_graph()
    
    # Initialize state
    initial_state = FitnessAppState(
        height=None,
        weight=None,
        age=None,
        gender=None,
        activity_level=None,
        goal_type=None,
        target_weight=None,
        target_days=None,
        user_notes=None,
        nutrition_plan=None,
        nutrition_plan_approved=None,
        workout_plan=None,
        current_step="start",
        user_input_pending=False,
        edit_request=None,
        chat_messages=[],
        chat_query=None,
        chat_response=None,
        error_message=None
    )
    
    # Configuration for conversation persistence
    config = RunnableConfig(configurable={"thread_id": "user_123"})
    
    # Example: Start the workflow
    print("🚀 Starting Fitness AI Application")
    print("=" * 50)
    
    # Step 1: Collect personal info
    result = app.invoke(initial_state, config)
    print(f"Current step: {result['current_step']}")
    
    # Example of updating state with user input
    updated_state = result.copy()
    updated_state.update({
        "height": 175.0,
        "weight": 80.0,
        "age": 28,
        "gender": "male",
        "activity_level": "moderately_active",
        "user_input_pending": False
    })
    
    # Continue workflow...
    print("\n" + "=" * 50)
    print("Workflow created successfully!")
    print("Key features:")
    print("✅ Sequential data collection")
    print("✅ AI-powered plan generation")
    print("✅ Interactive editing capability")
    print("✅ Contextual chat support")
    print("✅ State persistence")
    print("✅ Error handling")

if __name__ == "__main__":
    main()

🚀 Starting Fitness AI Application
📋 Please provide your personal information:
- Height (in cm)
- Weight (in kg)
- Age
- Gender (male/female/other)
- Activity Level (sedentary/lightly_active/moderately_active/very_active/extremely_active)
🎯 Please provide your fitness goals:
- Goal Type (weight_loss/muscle_gain/maintenance/strength/endurance)
- Target Weight (in kg)
- Target Timeline (in days)
📝 Please provide any additional notes (dietary restrictions, preferences, medical conditions, etc.):
This is optional but helps create a more personalized plan.
🍎 Generating your personalized nutrition plan...


AttributeError: 'NoneType' object has no attribute 'lower'